In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

# <center>App Store</center>

In [ ]:
import requests
import copy
import ipywidgets as ipw
from collections import Counter, OrderedDict
from os import path
from app import AppBase
from IPython.display import clear_output

In [ ]:
from config import aiidalab_apps

In [ ]:
class AppStoreControl(ipw.HBox):
    def __init__(self,):
        registry_url = 'https://aiidalab.materialscloud.org/appsdata/apps_meta.json'
        try:
            self.registry_sorted_list = sorted(requests.get(registry_url).json()['apps'].items())
        except ValueError:
            print("Registry server is unavailable! Can't load the apps")
            self.registry_sorted_list = []

        self.output = ipw.Output()
        self.items_per_page = ipw.BoundedIntText(value=10,
                                                 min=5,
                                                 max=40,
                                                 step=5,
                                                 description='Apps per page:',
                                                 disabled=False,
                                                 style = {'description_width': 'initial'},
                                                 layout={'width': '150px'},
                                                )
        self.group_filter = ipw.SelectMultiple(
            options=[],
            value=[],
            #rows=10,
            description='Filter groups',
            disabled=False
        )
        self.apply_group_filter = ipw.Button(description="apply")
        self.apply_group_filter.on_click(self.change_vis_list)
        self.items_per_page.observe(self.change_page_selector, names='value')

        self.page_selector = ipw.ToggleButtons(
            options=[],
            description='Page:',
            disabled=False,
            style={'button_width' : '30px'},
        )
        self.page_selector.observe(self.render, names='value')
        
        self.only_installed = ipw.Checkbox(
            value=False,
            description='Show only installed',
            disabled=False)
        self.only_installed.observe(self.change_vis_list, names='value')
        self.apps_to_display = [ AppBase(name, app, aiidalab_apps) for name, app in self.registry_sorted_list ]
        
        self.groups_counter = Counter([group for app in self.apps_to_display for group in app.groups])
        self.group_filter.options =  [key for key in self.groups_counter]
        self.change_page_selector()
        return super(AppStoreControl, self).__init__([self.only_installed, ipw.VBox([self.group_filter,
                                                                                    self.apply_group_filter
                                                                                    ])])
    
    def change_page_selector(self, b=None):
        initial_page = self.page_selector.value
        self.page_selector.options = range(1, len(self.apps_to_display) / self.items_per_page.value + 2)
        if initial_page == self.page_selector.value: # if page number changed in the previous step - the render
                                                     # will run automatically
                                                     # if not -it needs to be done manually
            self.render()

    def change_vis_list(self, b=None):                    
        self.apps_to_display = [ AppBase(name, app, aiidalab_apps) for name, app in self.registry_sorted_list ]            

        if self.only_installed.value:
            self.apps_to_display = [ app for app in self.apps_to_display if app.is_installed() ]
            
        if self.group_filter.value: 
            all_apps = self.apps_to_display
            self.apps_to_display = [] # create an array that contains all the apps
            self.app_corresponding_group = [] # create a parallel array that contains corresponding group names

            # iterate over all groups
            for group in self.group_filter.value:
                apps_belonging_to_group = [ app for app in all_apps if app.in_group(group) ]
                self.apps_to_display += apps_belonging_to_group
                self.app_corresponding_group += [group] + [None] * (len(apps_belonging_to_group) - 1)
            
        self.change_page_selector()

    def render(self, b=None):
        max_len = 180
        self.output.clear_output()
        page = self.page_selector.value - 1
        start = page * self.items_per_page.value
        end = (page + 1) * self.items_per_page.value
        with self.output:
            for number, app_base in enumerate(self.apps_to_display[start:end]):
                if self.group_filter.value and self.app_corresponding_group[start:end][number]:
                    display(ipw.HTML("<h1>{}</h1>".format(self.app_corresponding_group[start:end][number].title())))
                hr = ipw.HTML('<hr>')
                description = ipw.HTML("""<h2 style="text-align:center;">{}</h2>
                <div style="font-size: 15px; font-style: italic">Description: {}</div>
                <br>
                <div style="text-align:right;">{}</div>
                """.format(app_base.title,
                           app_base.description if len(app_base.description) < max_len else app_base.description[:max_len] + '...',
                           app_base.more))
                description.layout = {'width': '600px'}
                description.layout.margin = "0px 0px 0px 40px"
                result = ipw.VBox([hr,
                                   ipw.HBox([app_base.logo, description]),
                                   ipw.HBox([app_base.uninstall_button, app_base.update_button, app_base.install_button]),
                                   ipw.HBox([app_base.install_info]),
                                  ])
                display(result)
        
        return self.output

In [ ]:
control = AppStoreControl()
display(ipw.HTML("""/<a href="start.ipynb">home</a>/appstore"""))
display(control)
display(ipw.HBox([control.items_per_page, control.page_selector]))
display(control.render())
display(ipw.HBox([control.items_per_page, control.page_selector]))